In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep, time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import timeit

In [2]:
geckodriver_path = r"C:\Users\alkan\OneDrive\Masaüstü\twitter\driver\geckodriver.exe"

In [3]:
driver = webdriver.Firefox()
driver.get('https://twitter.com/login')

In [225]:
UserNames = []
Contents = []
Likes = []
Replies = []
Retweets = []
Bookmarks = []
Views = []
IsBlue = []

In [5]:
def parse_number(number_str):
    # Önce 'K' veya 'M' ile biten sayıları işleyin
    multiplier = 1
    if 'K' in number_str:
        multiplier = 1000
        number_str = number_str.replace('K', '')
    elif 'M' in number_str:
        multiplier = 1000000
        number_str = number_str.replace('M', '')
    
    # Noktayı kaldırın ve sayıyı çarpın
    number_str = number_str.replace('.', '')
    return int(float(number_str) * multiplier)

In [257]:
start_time = time()

# Sayfanın altına kaydırmak için bir fonksiyon
def go_under_page(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Yeni tweet'lerin yüklenmesini beklemek için bir fonksiyon
def wait_for_tweets(driver, timeout=10):
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )

# Toplanan tweet içeriklerinin sayısını kontrol etmek için bir sayaç
total_tweets = 0

# İlk 20 tweet'in içeriğini toplamak için bir döngü
while total_tweets < 20:
    # Sayfadaki tüm tweet 'article'larını bul
    articles = WebDriverWait(driver, 10).until(
        EC.visibility_of_all_elements_located((By.XPATH, "//article[@data-testid='tweet']"))
    )
    
    for article in articles:
        # Eğer 20 tweet'e ulaşıldıysa döngüyü durdur
        if total_tweets >= 20:
            break
        
        # Her bir tweet içeriğini bul ve listeye ekle
        try:
            #içerik
            content = article.find_element(By.XPATH, ".//div[@data-testid='tweetText']").text.replace('\n', '')
            Contents.append(content)
            
            # Yorum sayısını bul ve listeye ekle
            reply = article.find_element(By.XPATH, ".//div[@data-testid='reply']").text
            if reply == "":
                Replies.append(0)
            else:
                Replies.append(parse_number(reply))

            
            #Retweet'ler
            retweet = article.find_element(By.XPATH, ".//div[@data-testid= 'retweet']").text
            if not retweet:
                Retweets.append(0)
            else:
                Retweets.append(retweet)
    
            #Beğeni
            like = article.find_element(By.XPATH, ".//div[@data-testid='like']").text
            if like == "":
                Likes.append(0)
            else:
                Likes.append(parse_number(like))

            #Bookmarks
            bookmark = article.find_element(By.XPATH, ".//div[@data-testid='bookmark']").text
            if not bookmark:
                Bookmarks.append(0)
            else:
                Bookmarks.append(parse_number(bookmark))

            #Görüntülenme
            group = article.find_element(By.XPATH, ".//div[@role = 'group']").text
            split_group = group.split("\n")
            if not group:
                Views.append(0)
            else:
                parsed_numbers = [parse_number(element) for element in split_group]
                view = max(parsed_numbers)
                Views.append(view)
                         
            #UserName
            username = article.find_element(By.XPATH, ".//div[@data-testid='User-Name']")
            username_text = username.text
            username_split = username_text.split("\n")
            uniqusername = username_split[1]
            UserNames.append(uniqusername)
            username_id = username.get_attribute("id")
            
            try:
                blue = WebDriverWait(driver, 0.8).until(
                    EC.visibility_of_element_located((By.XPATH, f"//div[@id='{username_id}']//span[@class='css-1qaijid r-bcqeeo r-qvutc0 r-poiln3 r-1awozwy r-xoduu5']")))
                IsBlue.append(True)  # Hesap mavi ise True
            except:
                IsBlue.append(False)  # Hesap mavi değilse False 

            total_tweets += 1
        except Exception as e:
            print(f"Bir hata oluştu: {e}")
    
    # Eğer daha fazla tweet yüklenmesi gerekiyorsa sayfayı aşağı kaydır
    if total_tweets < 20:
        go_under_page(driver)
        wait_for_tweets(driver)

# Sürecin bitiş zamanını kaydet ve süreyi hesapla
end_time = time()
duration = end_time - start_time

print(f"Süreç {duration:.2f} saniye sürdü.")


Süreç 30.67 saniye sürdü.


In [271]:
import pandas as pd

In [281]:
df = pd.DataFrame({
    'UserNames': UserNames,
    'Contents': Contents,
    'Likes': Likes,
    'Replies': Replies,
    'Retweets': Retweets,
    'Bookmarks': Bookmarks,
    'Views': Views,
    'IsBlue': IsBlue
})

In [283]:
df

,UserNames,Contents,Likes,Replies,Retweets,Bookmarks,Views,IsBlue
0,@eltandexpert,Sana #kırmızı çok yakışıyor #Türkiyem #ankara #İstanbul #mansuryavas #ekremimamoglu #secim2024,1,0,1,0,606,False
1,@haberinizoldu,Dilek İmamoğlu eşi Ekrem İmamoğlu’yla olan fotoğrafını ‘’Sevgi ve iyilik kazandı!’’ notuyla paylaştı. #secim2024,1,0,0,0,427,False
2,@andveeund,Kürtlerin terör seviciliği yine şaşırtmadı. #secim2024,0,0,0,0,316,False
3,@tacieksikkral,Yarın Nas kuralları devreye girer. Faiz %5 olur. Kutlamanın bokunu çıkarmayın.#Seçim2024,2,0,0,0,302,False
4,@umrumda_degiill,Bu geceye iyi gider#secim2024,17,1,0,0,445,False
5,@TANLGN,"Ekrem İmamoğlu: @ekrem_imamoglu - ""11 belediyeyi daha kazanmış görünüyoruz.""1 milyonun üzerinde farkla öndeyiz. @herkesicinCHP#Seçim2024",0,0,0,0,288,False
6,@Akcyys,#secim2024Akparti Ankara il teşkilatı zaten başarısı ortada. Ama mağdur olan halk için üst mercilere ilettiğimizde bakın yanlıştan dönün dediğimizde görmez iseniz bugün okursunuz hepsini.Eş dost herkesi uyardık ümitlenmeyin ama bu kadar da fark olacak ne yaptınız vatandaşa!,0,0,0,0,433,False
7,@Kkdp1907,Lan şaka demesin bunlar #secim2024,0,0,0,0,288,False
8,@kesintihaber,Ekrem İmamoğlu | 1 milyon farkla öndeyiz#secim2024 #seçim #İstanbul #secimİstanbul,1,0,0,0,282,False
9,@yazici4692,Benim #secim2024 ten anladığım;M/illetin derdi büyük. Baştakilere kızgın ve kırgın.,2,0,0,0,292,False
